### Standard & Poor's (S&P 500 index):
- Preprocessing.
- Preparation and Visualization.

### Importing Libraries 

In [1]:
import pandas as pd
import yfinance as yf
import numpy as np # for numerical operations
import seaborn as sns #visualisation
import matplotlib.pyplot as plt # visualization
%matplotlib inline 
import matplotlib.ticker as ticker # Library to customize ticks
from datetime import date
import holidays

import warnings # filter warnings
warnings.filterwarnings('ignore')

The selected source for the experiment is Yahoo Finance, through the integration of the yfinance library and jupyter.
- Timeframe selected for the project is 10 years (2014-2023).
- The frequency to be used is business day, the selection is because the market is open during these days.
- The period include real world scenarios such as Brexit in 2016 and Covid-19.
- Is also believed that in 10 years of data, volatility and trends are present in the market. 

### Importing dataset from yahoo finance (yfinance library).

In [2]:
# Defines ticker symbol for S&P 500 index.
ticker_symbol = "^GSPC"

# Fetch historical data.
sp500_data = yf.download(ticker_symbol, start="2014-01-01", end="2023-12-31")

[*********************100%%**********************]  1 of 1 completed


The command display provides an overview of the dataset: 
- Head: First five rows.
- Tail: Last five rows.
- Shape: Number of rows and columns.

### Data PreProcessing 

In [3]:
display(sp500_data)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-01-02,1845.859985,1845.859985,1827.739990,1831.979980,1831.979980,3080600000
2014-01-03,1833.209961,1838.239990,1829.130005,1831.369995,1831.369995,2774270000
2014-01-06,1832.310059,1837.160034,1823.729980,1826.770020,1826.770020,3294850000
2014-01-07,1828.709961,1840.099976,1828.709961,1837.880005,1837.880005,3511750000
2014-01-08,1837.900024,1840.020020,1831.400024,1837.489990,1837.489990,3652140000
...,...,...,...,...,...,...
2023-12-22,4753.919922,4772.939941,4736.770020,4754.629883,4754.629883,3046770000
2023-12-26,4758.859863,4784.720215,4758.450195,4774.750000,4774.750000,2513910000
2023-12-27,4773.450195,4785.390137,4768.899902,4781.580078,4781.580078,2748450000


- The dataset is composed by 2516 rows and 6 features. In this present project the Close feature will be the focus of the study, representing the closing index at the end of the trading day.

In [4]:
# Check for missing values
missing_values = sp500_data.isnull().sum()

# Print the number of missing values for each column
print("Missing values:")
print(missing_values)

Missing values:
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64


- The S&P 500 index dataset imported from yfinance is composed by 2516 observations and 6 features.

The dataset does not present outliers at this point. However, by observind the first five rows it is noticeable that there are not five days in a row (business days), therefore holidays on weekdays might have been excluded from the set due to no trading activity.

In [5]:
# Define start and end dates - 10 years period.
start_date = '2014-01-01'
end_date = '2023-12-31'

# Generate business days between the start and end dates
business_days = pd.bdate_range(start=start_date, end=end_date)

# Get the total number of business days
total_business_days = len(business_days)

print("Total number of business days between", start_date, "and", end_date, "is:", total_business_days)

Total number of business days between 2014-01-01 and 2023-12-31 is: 2608


The calendar within the selected period has 2608 business days, having more days than the number of rows in the dataset, providing more evidence for the exclusion of holidays on weekdays (no trading). 
- weekends could be considered noise (market is closed).
- by analysing just the business days it can be focused on days that the market is actually open, reducing the amount of noise which could affect the model performance. 

#### <b> After the models have being implemented a second round of models can be applied using daily frequency, which will include all the seven days (addition not only of holidays on weekdays, but also, weekends).

- Check the performance of the models and compare what frequency techniques performs better or worse. In addition, check if the introduction of noise (handling missing values) affected the models and justify even deeper the choice for the frequency.

In [16]:
# Defining start and end dates (ten years timeframe)
start_date = date(2014, 1, 1)
end_date = date(2023, 12, 31)

# Get all US business holidays between start_date and end_date
us_holidays = holidays.US()

# Create a complete date range covering all weekdays
all_weekdays = pd.date_range(start=start_date, end=end_date, freq='B')

# Filter out holidays that fall on weekdays
us_business_holidays = [holiday for holiday in us_holidays.keys() if holiday in all_weekdays]

# Filter out weekends
weekdays = all_weekdays.difference(pd.to_datetime(us_business_holidays))

# Create DataFrame with all weekdays
weekdays_df = pd.DataFrame({'Date': weekdays})

# Set 'Date' column as index
weekdays_df.set_index('Date', inplace=True)

# Create DataFrame with S&P 500 data
df = pd.DataFrame(sp500_data, columns=['Open', 'Close'])

# Merge the two DataFrames on the index
merged_df = weekdays_df.merge(df, left_index=True, right_index=True, how='left')

# Convert back to Series
df_BF = merged_df

# Display the updated Series
display(df_BF)

,Open,Close
Date,,
2014-01-01,NaN,NaN
2014-01-02,1845.859985,1831.979980
2014-01-03,1833.209961,1831.369995
2014-01-06,1832.310059,1826.770020
2014-01-07,1828.709961,1837.880005
...,...,...
2023-12-25,NaN,NaN
2023-12-26,4758.859863,4774.750000
2023-12-27,4773.450195,4781.580078


- After the inputation of the holidays the number of entries match with the total (2608), which was generated by the previous code.
- The inputed missing days were added with NaN values, which will have to be addressed. 

In [17]:
# Check for missing values
missing_values = df_BF.isnull().sum()

# Print the number of missing values for each column
print("Missing values:")
print(missing_values)

Missing values:
Open     92
Close    92
dtype: int64


During this 10 years period, the market had 92 holidays on business days.
<b> Assuming that technically the close value of the last trading day will maintain untill the following trading day.
- The technique to be performed is the foward fill, which will use close prices of the day before (last day that it was open).
- Second stage will be performed using the back fill for the first data point in the set, since there is not a close index from the previous day, the open value from the previous trading day will be used.  

In [9]:
# Forward fill NaN values in the 'Close' column
df_BF.fillna(method='ffill', inplace=True)

# Display the updated DataFrame with forward filled values
display(df_BF)

Date
2014-01-01            NaN
2014-01-02    1831.979980
2014-01-03    1831.369995
2014-01-06    1826.770020
2014-01-07    1837.880005
                 ...     
2023-12-25    4754.629883
2023-12-26    4774.750000
2023-12-27    4781.580078
2023-12-28    4783.350098
2023-12-29    4769.830078
Name: Close, Length: 2608, dtype: float64

In [10]:
# Get the 'Open' price from the next available trading day (02/01/14) in sp500_data
next_trading_day_open = sp500_data.loc['2014-01-02', 'Open']

# Fill the missing value in df_BF with the 'Open' price from the next trading day
df_BF.loc['2014-01-01'] = next_trading_day_open

In [11]:
# Display the updated DataFrame with the first business/holiday of the year. 
display(df_BF)

Date
2014-01-01    1845.859985
2014-01-02    1831.979980
2014-01-03    1831.369995
2014-01-06    1826.770020
2014-01-07    1837.880005
                 ...     
2023-12-25    4754.629883
2023-12-26    4774.750000
2023-12-27    4781.580078
2023-12-28    4783.350098
2023-12-29    4769.830078
Name: Close, Length: 2608, dtype: float64

In [12]:
# Check for missing values
missing_values = df_BF.isnull().sum()

# Print the number of missing values for each column
print("Missing values:")
print(missing_values)

Missing values:
0


In [13]:
sp500_data['Close'].describe()

count    2516.000000
mean     3005.883140
std       901.337964
min      1741.890015
25%      2124.267517
50%      2798.160034
75%      3907.079956
max      4796.560059
Name: Close, dtype: float64

In [14]:
df_BF.describe()

count    2608.000000
mean     3005.915583
std       901.622414
min      1741.890015
25%      2124.267517
50%      2795.005005
75%      3903.642578
max      4796.560059
Name: Close, dtype: float64

Looking at the statistical properties of the S&P500 data (withouy any imputations) and the df_BF (with imputations), it is observable that the properties barely changed, the biggest difference are in the 50% (-3.16) and 75% (-3.44). However, taking into consideration the scale the change is not significant. 

The statistical features of the imputed dataframe, the df_BF:
- The variable ranges from 1741.89 to 4796.56
- It has a mean (average) of 3005.92.
- The standard deviation is 901.62, and in the present context can be considered high because it represents almost 30% of the avg value, but also, taking into consideration the scale of the min and max values. 

The stock market prices are kept as the same value as the closing prices of the last day that was trading. Thus, the technique to be performed is the foward fill, inputing the close value from the day before.

- Other imputation techniques can be applied later to check the results and implications of each approach to handle the missing values.

### Outliers and Skewness

In [ ]:
plt.figure(figsize=(12, 8)) # Set up the plot and adjust the figure size.
sns.set(style="whitegrid", font_scale=1.2) # Add a whitegrid in the backgound for clear visualization.
# Creates the boxplot with all the columns.
ax = sns.boxplot(data=sp500_data['Close'], width=0.5) # Sets the width to 0.5
 # Add labels and ha argument used to align the labels to the designated boxplot.
ax.set_xlabel('Data') # Adds x-axis label.
ax.set_ylabel('Value') # Adds y-axis label.
ax.set_title('Boxplot of S&P500 index overtime (10 years)') # Sets title to tle boxplot.

plt.tight_layout() # Adjusts the layout.
plt.show() # Shows the plot.

- The .describe along with the boxplot shows that the mean is above the median indicating a skewness towards higher values.
- According to the boxplot, the close variable does not present any values outside of the skewers, therefore no outliers. In addition, IQR (Inter Quartile Range) and MAD (Median Absolute Deviation) will be performed to be sure.

In [ ]:
Q1 = sp500_data['Adj Close'].quantile(0.25)
Q3 = sp500_data['Adj Close'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
outliers = sp500_data['Close'][(sp500_data['Close'] < lower_bound) | (sp500_data['Close'] > upper_bound)]

# Check if there are outliers
if outliers.empty:
    print("No outliers detected.")
else:
    print("Outliers detected:")
    print(outliers)

In [ ]:
# Calculate the median absolute deviation (MAD)
median = sp500_data['Close'].median()
mad = np.median(np.abs(sp500_data['Close'] - median))

# Define a threshold for outliers (e.g., 3 times the MAD)
threshold_mad = 3 * mad

# Identify outliers using the MAD method
outliers_mad = sp500_data['Close'][np.abs(sp500_data['Close'] - median) > threshold_mad]

# Display outliers detected by the MAD method
print("Outliers detected using MAD method:")
print(outliers_mad)

In [ ]:
from scipy import stats

# Detect outliers using Z-score method
z_scores = stats.zscore(sp500_data['Close'])
outliers = (z_scores > 3) | (z_scores < -3)
outlier_indices = sp500_data.index[outliers]
print('Outlier indices:', outlier_indices)


- The previous lines of code shows that no outliers are present. 

### Distribution of Close index

In [ ]:
# Distribution plot of Close prices
plt.figure(figsize=(10, 6))
sns.histplot(sp500_data['Close'], bins=20, kde=True, color='blue')
plt.title('Distribution of S&P500 Index')
plt.xlabel('Close S&P500 Index')
plt.ylabel('Frequency')
plt.show()

The close index presents a multimodal distribution of values. 

In [ ]:
# Plotting
plt.figure(figsize=(12, 8))

# Time Series Plot of Close Price
plt.plot(sp500_data.index, sp500_data['Close'], color='blue')
plt.title('S&P 500 Close Price Over Time')
plt.xlabel('Date')
plt.ylabel('Close Price')

plt.show()

In [ ]:
from statsmodels.tsa.stattools import adfuller

# Perform Augmented Dickey-Fuller test for stationarity
adf_test = adfuller(sp500_data['Close'])
print('ADF Statistic:', adf_test[0])
print('p-value:', adf_test[1])

- Based on the p-value of 0.92, which is way above 0.05, the time series is likely non-stationary. Thus, additional techniques such as differencing, detrending approaches will have to be applied to attend auto regressive requirements. 

In [ ]:
# Calculates daily stock returns.
sp500_data['Daily stock_return'] = sp500_data['Adj Close'].pct_change()

# Prints the sp500_data with the new column (the stock returns).
print(sp500_data)

In [ ]:
# Calculate the minimum and maximum values in the column
min_value = sp500_data['Daily stock_return'].min()
max_value = sp500_data['Daily stock_return'].max()

# Print the minimum and maximum values
print("Minimum value:", min_value)
print("Maximum value:", max_value)

### Timeframe selected for the second section is from 01/01/00 to 31/12/23

In [ ]:
# Fetch historical data
df = yf.download(ticker_symbol, start="2000-01-01", end="2023-12-31")

In [ ]:
display(df)

In [ ]:
# Check for missing values
missing_values = df.isnull().sum()

# Print the number of missing values for each column
print("Missing values:")
print(missing_values)

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
plt.hist(df['Adj Close'], bins=20, color='skyblue', edgecolor='black')
plt.title('Distribution of S&P 500 Adj Closing Prices')
plt.xlabel('Close Price')
plt.ylabel('Frequency')
plt.show()

- The Adjusted Close values are highly right-skewed. 

The following cells will check for outliers and experimentation of techniques.

In [ ]:
# Boxplot, a visualization technique that uses the IQR method. 

plt.figure(figsize=(12, 8)) # Set up the plot and adjust the figure size.
sns.set(style="whitegrid", font_scale=1.2) # Add a whitegrid in the backgound for clear visualization.
# Creates the boxplot with all the columns.
ax = sns.boxplot(data=df[['Open','High','Low','Close','Adj Close']], width=0.5) # Sets the width to 0.5
 # Add labels and ha argument used to align the labels to the designated boxplot.
ax.set_xlabel('Data') # Adds x-axis label.
ax.set_ylabel('Value') # Adds y-axis label.
ax.set_title('Boxplot of S&P500 index overtime (from 01/00)') # Sets title to tle boxplot.

plt.tight_layout() # Adjusts the layout.
plt.show() # Shows the plot.

In [ ]:
Q1 = df['Adj Close'].quantile(0.25)
Q3 = df['Adj Close'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
outliers = df['Adj Close'][(df['Adj Close'] < lower_bound) | (df['Adj Close'] > upper_bound)]

# Check if there are outliers
if outliers.empty:
    print("No outliers detected.")
else:
    print("Outliers detected:")
    print(outliers)

In [ ]:
# Looking for outliers using the percentile method (0.05 - 0.95)

# Define the lower and upper percentile thresholds
lower_percentile = df['Adj Close'].quantile(0.05)
upper_percentile = df['Adj Close'].quantile(0.95)

# Identify outliers
outliers_percentile = df[(df['Adj Close'] < lower_percentile) | (df['Adj Close'] > upper_percentile)]

# Display outliers
print("Outliers detected using Percentile method:")
print(outliers_percentile)


#### Because of the distribution of the data, the percentile technique presented way more outliers than the IQR. Therefore, the chosen approach will be the IQR, which is more appropriate when the data is skewed. 

In [ ]:
# Not appropriate, the data is not normally distributed so it could provide wrong insights regarding outliers.

#from scipy.stats import zscore
#df['Z_score'] = zscore(df['Adj Close'])
#outliers = df[abs(df['Z_score']) > 3]

# Check if there are outliers
#if outliers.empty:
#    print("No outliers detected.")
#else:
#    print("Outliers detected:")
#    print(outliers)

In [ ]:
# Calculate the median absolute deviation (MAD)
median = df['Adj Close'].median()
mad = np.median(np.abs(df['Adj Close'] - median))

# Define a threshold for outliers (e.g., 3 times the MAD)
threshold_mad = 3 * mad

# Identify outliers using the MAD method
outliers_mad = df[np.abs(df['Adj Close'] - median) > threshold_mad]

# Display outliers detected by the MAD method
print("Outliers detected using MAD method:")
print(outliers_mad)


In [ ]:
# Plotting
plt.figure(figsize=(12, 8))

# Time Series Plot of Close Price
plt.plot(df.index, df['Adj Close'], color='blue')
plt.title('S&P 500 Close Price Over Time')
plt.xlabel('Date')
plt.ylabel('Close Price')

plt.show()

In [ ]:
# Calculates daily stock returns.
df['Daily stock_return'] = df['Adj Close'].pct_change()

# Prints the df with the new column (the stock returns).
print(df)

In [ ]:
# Calculate the minimum and maximum values in the column
min_value = df['Daily stock_return'].min()
max_value = df['Daily stock_return'].max()

# Print the minimum and maximum values
print("Minimum value:", min_value)
print("Maximum value:", max_value)

In [ ]:
plt.figure(figsize=(12, 8)) # Set up the plot and adjust the figure size.
sns.set(style="whitegrid", font_scale=1.2) # Add a whitegrid in the backgound for clear visualization.
# Creates the boxplot with all the columns.
ax = sns.boxplot(data=df['Daily stock_return'], width=0.5) # Sets the width to 0.5
 # Add labels and ha argument used to align the labels to the designated boxplot.
ax.set_xlabel('Data') # Adds x-axis label.
ax.set_ylabel('Value') # Adds y-axis label.
ax.set_title('Boxplot of S&P500 index overtime (from 01/00)') # Sets title to tle boxplot.

plt.tight_layout() # Adjusts the layout.
plt.show() # Shows the plot.